In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

In [5]:
Title = []
#Category = []
Price = []
Rating = []
ReviewsCount = []
StudentCount = []
totla_length = []
LectureCount = []
Discount = []
InstructorName = []
InstructorRtaing = []

In [6]:
url = 'https://www.udemy.com/courses/it-and-software/'
r = requests.get(url)

In [47]:
soup = BeautifulSoup(r.content,'html.parser')

In [48]:
soup


<!DOCTYPE html>

<html lang="en-us">
<head>
<title>
    Online IT &amp; Software Courses
 | Udemy</title>
<meta content="Study the latest operating system architectures or prepare for certification exams with courses from Udemy's world-class instructors." name="description"/><meta content="mult" name="medium"/><meta content="summary_large_image" name="twitter:card"/><meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/><meta content="Online IT &amp; Software Courses" name="title"/><meta content="app-id=562413829, affiliate-data=ct=Safari_SmartBanner&amp;amp;pt=1240482" name="apple-itunes-app"/><meta content="app-id=com.udemy.android" name="google-play-app"/><meta content="313137469260" property="fb:app_id"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="Online IT &amp; Software Courses" prop

In [ ]:
for name in products.find_all('a',class_='absolute-link product-link'):
        Category.append(c)
        Name.append(name.get('title'))

In [43]:
courses = soup.find_all('div', class_='list-menu--list-menu-container--21IlT')

In [44]:
courses

[<div class="list-menu--list-menu-container--21IlT browse-nav--nav-container--2QiNy"><div class="js-browse-nav-level-one browse-nav--nav--1WzbY"><ul class="unstyled-list udlite-block-list list-menu--section--BZ3j9"><li><div class="list-menu--item--1crtM udlite-block-list-item udlite-block-list-item-small udlite-block-list-item-neutral udlite-text-sm"><div class="udlite-block-list-item-content udlite-block-list-item-content-loading"> </div></div></li><li><div class="list-menu--item--1crtM udlite-block-list-item udlite-block-list-item-small udlite-block-list-item-neutral udlite-text-sm"><div class="udlite-block-list-item-content udlite-block-list-item-content-loading"> </div></div></li><li><div class="list-menu--item--1crtM udlite-block-list-item udlite-block-list-item-small udlite-block-list-item-neutral udlite-text-sm"><div class="udlite-block-list-item-content udlite-block-list-item-content-loading"> </div></div></li><li><div class="list-menu--item--1crtM udlite-block-list-item udlite

In [50]:
import requests

s = requests.Session()
cookies = s.get('https://www.udemy.com').cookies
headers={"Referer": "https://www.udemy.com/courses/search/?q=python&skip_price=false"}

In [51]:
data = s.get('https://www.udemy.com/api-2.0/search-courses/?p={}&q=python&skip_price=false'.format(page_counter), cookies=cookies, headers=headers).json()
data

{'count': 10000,
 'has_courses_for_org': True,
 'courses': [{'predictive_score': 5.211237,
   'instructional_level_simple': 'Beginner',
   'objectives_summary': ['Learn how to solve real life problem using the Linear and Logistic Regression technique',
    'Preliminary analysis of data using Univariate and Bivariate analysis before running regression analysis',
    'Understand how to interpret the result of Linear and Logistic Regression model and translate them into actionable insight'],
   'is_in_user_subscription': False,
   'tracking_id': 'nVOBRNJgTXyfcvJGWECmbQ',
   'image_240x135': 'https://img-c.udemycdn.com/course/240x135/3202936_31f0_4.jpg',
   'relevancy_score': 11729.359,
   'badges': [],
   'image_100x100': 'https://img-c.udemycdn.com/course/100x100/3202936_31f0_4.jpg',
   'is_paid': True,
   'visible_instructors': [{'display_name': 'Start-Tech Academy',
     'title': 'Start-Tech Academy'}],
   'learn_url': '/course/linear-regression-and-logistic-regression-in-python-startt

In [55]:
for page_counter in range(1, 10):
    data = s.get('https://www.udemy.com/api-2.0/search-courses/?p={}&q=python&skip_price=false'.format(page_counter), cookies=cookies, headers=headers).json()
    for course in data['courses']:
        params = {'course_ids': [str(course['id']),],
              'fields/[pricing_result/]': ['price',]}
        Title.append(course['title'])
        price = s.get('https://www.udemy.com/api-2.0/pricing/', params=params, cookies=cookies).json()['courses'][str(course['id'])]['price']['amount']
        print({'title': title, 'price': price})

{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 12.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 12.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 18.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 18.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 12.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 18.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 21.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 12.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 16.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 19.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 16.99}
{'title': 

{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 12.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 12.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 12.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 12.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 12.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 12.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 12.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 12.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 15.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 13.99}
{'title': 'Complete Python 3 Programming Bootcamp: Beginner to Advanced', 'price': 15.99}
{'title': 

In [56]:
Title

['Python for Finance and Algorithmic Trading with QuantConnect',
 'A deep understanding of deep learning (with Python intro)',
 'The Python Mega Course: Build 10 Real World Applications',
 '100 Days of Code - The Complete Python Pro Bootcamp for 2021',
 'Learn Python Programming Masterclass',
 'Automate the Boring Stuff with Python Programming',
 '2021 Complete Python Bootcamp From Zero to Hero in Python',
 'Python for Data Science and Machine Learning Bootcamp',
 'Python for Beginners: Learn Python Programming (Python 3)',
 'Complete Python Developer in 2021: Zero to Mastery',
 'Machine Learning A-Z™: Hands-On Python & R In Data Science',
 'REST APIs with Flask and Python',
 'Machine Learning, Data Science and Deep Learning with Python',
 'The Python Bible™ | Everything You Need to Program in Python',
 'The Complete Python Course | Learn Python by Doing',
 'The Modern Python 3 Bootcamp',
 'Python for Finance: Investment Fundamentals & Data Analytics',
 'Python for beginners',
 'Data A